In [1]:
import time
import scipy.sparse as sp
import numpy as np
import tensorflow as tf
import argparse
from tf.gat.models import SpGAT
from tf.gat.utils import process
from utils import *
from load_ether import load_data

/home/ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [2]:
checkpt_file = 'ether_pre_trained.ckpt'

dataset = 'ether'

# training params
batch_size = 1
nb_epochs = 100000
patience = 100
lr = 0.005  # learning rate
l2_coef = 0.0005  # weight decay
hid_units = [8] # numbers of hidden units per each attention head in each layer
n_heads = [8, 1] # additional entry for the output layer
residual = False
nonlinearity = tf.nn.elu
# model = GAT
model = SpGAT
sparse = True

In [3]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data("./data/ether/")

features = features.toarray()
print("features = {} type = {}".format(features.shape, type(features)))
in_transaction_value = np.array(features[:, 0])
in_transaction_count = np.array(features[:, 1])
out_transaction_value = np.array(features[:, 3])
out_transaction_count = np.array(features[:, 4])
in_transaction_value = np.reshape(in_transaction_value, newshape=(in_transaction_value.shape[0], 1))
in_transaction_count = np.reshape(in_transaction_count, newshape=(in_transaction_count.shape[0], 1))
out_transaction_value = np.reshape(out_transaction_value, newshape=(out_transaction_value.shape[0], 1))
out_transaction_count = np.reshape(out_transaction_count, newshape=(out_transaction_count.shape[0], 1))
features = np.concatenate((in_transaction_value, in_transaction_count, out_transaction_value, out_transaction_count), axis=1)
print("features = {} type = {}".format(features.shape, type(features)))

features = sp.lil_matrix(features)
features, spars = process.preprocess_features(features)

nb_nodes = features.shape[0]
ft_size = features.shape[1]
nb_classes = y_train.shape[1]

features = features[np.newaxis]
y_train = y_train[np.newaxis]
y_val = y_val[np.newaxis]
y_test = y_test[np.newaxis]
train_mask = train_mask[np.newaxis]
val_mask = val_mask[np.newaxis]
test_mask = test_mask[np.newaxis]
biases = process.preprocess_adj_bias(adj)

features = (1402220, 12) type = <class 'numpy.ndarray'>
features = (1402220, 4) type = <class 'numpy.ndarray'>


In [4]:
with tf.Graph().as_default():
    with tf.name_scope('input'):
        ftr_in = tf.placeholder(dtype=tf.float32, shape=(batch_size, nb_nodes, ft_size))
        if sparse:
            bias_in = tf.sparse_placeholder(dtype=tf.float32)
        else:
            bias_in = tf.placeholder(dtype=tf.float32, shape=(batch_size, nb_nodes, nb_nodes))
        lbl_in = tf.placeholder(dtype=tf.int32, shape=(batch_size, nb_nodes, nb_classes))
        msk_in = tf.placeholder(dtype=tf.int32, shape=(batch_size, nb_nodes))
        attn_drop = tf.placeholder(dtype=tf.float32, shape=())
        ffd_drop = tf.placeholder(dtype=tf.float32, shape=())
        is_train = tf.placeholder(dtype=tf.bool, shape=())

    logits = model.inference(ftr_in, nb_classes, nb_nodes, is_train,
                                attn_drop, ffd_drop,
                                bias_mat=bias_in,
                                hid_units=hid_units, n_heads=n_heads,
                                residual=residual, activation=nonlinearity)
    log_resh = tf.reshape(logits, [-1, nb_classes])
    lab_resh = tf.reshape(lbl_in, [-1, nb_classes])
    msk_resh = tf.reshape(msk_in, [-1])
    loss = model.masked_softmax_cross_entropy(log_resh, lab_resh, msk_resh)
    accuracy = model.masked_accuracy(log_resh, lab_resh, msk_resh)

    train_op = model.training(loss, lr, l2_coef)

    saver = tf.train.Saver()

    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    vlss_mn = np.inf
    vacc_mx = 0.0
    curr_step = 0

    with tf.Session() as sess:
        sess.run(init_op)

        train_loss_avg = 0
        train_acc_avg = 0
        val_loss_avg = 0
        val_acc_avg = 0

        for epoch in range(nb_epochs):
            tr_step = 0
            tr_size = features.shape[0]

            while tr_step * batch_size < tr_size:
                if sparse:
                    bbias = biases
                else:
                    bbias = biases[tr_step*batch_size:(tr_step+1)*batch_size]

                _, loss_value_tr, acc_tr = sess.run([train_op, loss, accuracy],
                    feed_dict={
                        ftr_in: features[tr_step*batch_size:(tr_step+1)*batch_size],
                        bias_in: bbias,
                        lbl_in: y_train[tr_step*batch_size:(tr_step+1)*batch_size],
                        msk_in: train_mask[tr_step*batch_size:(tr_step+1)*batch_size],
                        is_train: True,
                        attn_drop: 0.6, ffd_drop: 0.6})
                train_loss_avg += loss_value_tr
                train_acc_avg += acc_tr
                tr_step += 1

            vl_step = 0
            vl_size = features.shape[0]

            while vl_step * batch_size < vl_size:
                if sparse:
                    bbias = biases
                else:
                    bbias = biases[vl_step*batch_size:(vl_step+1)*batch_size]
                loss_value_vl, acc_vl = sess.run([loss, accuracy],
                    feed_dict={
                        ftr_in: features[vl_step*batch_size:(vl_step+1)*batch_size],
                        bias_in: bbias,
                        lbl_in: y_val[vl_step*batch_size:(vl_step+1)*batch_size],
                        msk_in: val_mask[vl_step*batch_size:(vl_step+1)*batch_size],
                        is_train: False,
                        attn_drop: 0.0, ffd_drop: 0.0})
                val_loss_avg += loss_value_vl
                val_acc_avg += acc_vl
                vl_step += 1

            print('Training: loss = %.5f, acc = %.5f | Val: loss = %.5f, acc = %.5f' %
                    (train_loss_avg/tr_step, train_acc_avg/tr_step,
                    val_loss_avg/vl_step, val_acc_avg/vl_step))

            if val_acc_avg/vl_step >= vacc_mx or val_loss_avg/vl_step <= vlss_mn:
                if val_acc_avg/vl_step >= vacc_mx and val_loss_avg/vl_step <= vlss_mn:
                    vacc_early_model = val_acc_avg/vl_step
                    vlss_early_model = val_loss_avg/vl_step
                    saver.save(sess, checkpt_file)
                vacc_mx = np.max((val_acc_avg/vl_step, vacc_mx))
                vlss_mn = np.min((val_loss_avg/vl_step, vlss_mn))
                curr_step = 0
            else:
                curr_step += 1
                if curr_step == patience:
                    print('Early stop! Min loss: ', vlss_mn, ', Max accuracy: ', vacc_mx)
                    print('Early stop model validation loss: ', vlss_early_model, ', accuracy: ', vacc_early_model)
                    break

            train_loss_avg = 0
            train_acc_avg = 0
            val_loss_avg = 0
            val_acc_avg = 0

        saver.restore(sess, checkpt_file)

        ts_size = features.shape[0]
        ts_step = 0
        ts_loss = 0.0
        ts_acc = 0.0

        while ts_step * batch_size < ts_size:
            if sparse:
                bbias = biases
            else:
                bbias = biases[ts_step*batch_size:(ts_step+1)*batch_size]
            loss_value_ts, acc_ts = sess.run([loss, accuracy],
                feed_dict={
                    ftr_in: features[ts_step*batch_size:(ts_step+1)*batch_size],
                    bias_in: bbias,
                    lbl_in: y_test[ts_step*batch_size:(ts_step+1)*batch_size],
                    msk_in: test_mask[ts_step*batch_size:(ts_step+1)*batch_size],
                    is_train: False,
                    attn_drop: 0.0, ffd_drop: 0.0})
            ts_loss += loss_value_ts
            ts_acc += acc_ts
            ts_step += 1

        print('Test loss:', ts_loss/ts_step, '; Test accuracy:', ts_acc/ts_step)

        sess.close()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Training: loss = 2.18025, acc = 0.22552 | Val: loss = 1.70874, acc = 0.17073
Training: loss = 2.27202, acc = 0.18706 | Val: loss = 1.69309, ac

Training: loss = 1.36019, acc = 0.48427 | Val: loss = 1.28318, acc = 0.51220
Training: loss = 1.29918, acc = 0.47203 | Val: loss = 1.28199, acc = 0.51220
Training: loss = 1.35486, acc = 0.46329 | Val: loss = 1.28033, acc = 0.51220
Training: loss = 1.35596, acc = 0.47552 | Val: loss = 1.27894, acc = 0.51220
Training: loss = 1.34573, acc = 0.45804 | Val: loss = 1.27760, acc = 0.51220
Training: loss = 1.38751, acc = 0.42832 | Val: loss = 1.27608, acc = 0.51220
Training: loss = 1.38221, acc = 0.46678 | Val: loss = 1.27475, acc = 0.51220
Training: loss = 1.33652, acc = 0.51049 | Val: loss = 1.27357, acc = 0.51220
Training: loss = 1.40546, acc = 0.47552 | Val: loss = 1.27230, acc = 0.51220
Training: loss = 1.33143, acc = 0.44930 | Val: loss = 1.27109, acc = 0.51220
Training: loss = 1.28496, acc = 0.51748 | Val: loss = 1.26990, acc = 0.51220
Training: loss = 1.33177, acc = 0.48252 | Val: loss = 1.26913, acc = 0.51220
Training: loss = 1.35010, acc = 0.46329 | Val: loss = 1.26828, acc = 0.51220

Training: loss = 1.27102, acc = 0.49301 | Val: loss = 1.23361, acc = 0.51220
Training: loss = 1.28897, acc = 0.48427 | Val: loss = 1.23323, acc = 0.51220
Training: loss = 1.30172, acc = 0.50350 | Val: loss = 1.23383, acc = 0.51220
Training: loss = 1.33319, acc = 0.45979 | Val: loss = 1.23434, acc = 0.51220
Training: loss = 1.25076, acc = 0.50699 | Val: loss = 1.23487, acc = 0.51220
Training: loss = 1.23641, acc = 0.55594 | Val: loss = 1.23531, acc = 0.51220
Training: loss = 1.30092, acc = 0.49301 | Val: loss = 1.23570, acc = 0.51220
Training: loss = 1.22239, acc = 0.50874 | Val: loss = 1.23572, acc = 0.51220
Training: loss = 1.33194, acc = 0.51923 | Val: loss = 1.23609, acc = 0.51220
Training: loss = 1.27570, acc = 0.50350 | Val: loss = 1.23625, acc = 0.51220
Training: loss = 1.28992, acc = 0.48601 | Val: loss = 1.23631, acc = 0.51220
Training: loss = 1.27580, acc = 0.55070 | Val: loss = 1.23672, acc = 0.51220
Training: loss = 1.22522, acc = 0.52273 | Val: loss = 1.23719, acc = 0.51220

Training: loss = 1.20078, acc = 0.54021 | Val: loss = 1.22279, acc = 0.51220
Training: loss = 1.23055, acc = 0.51399 | Val: loss = 1.22225, acc = 0.51220
Training: loss = 1.20469, acc = 0.51573 | Val: loss = 1.22139, acc = 0.51220
Training: loss = 1.26080, acc = 0.48252 | Val: loss = 1.22068, acc = 0.51220
Training: loss = 1.30696, acc = 0.47552 | Val: loss = 1.22095, acc = 0.51220
Training: loss = 1.23937, acc = 0.51049 | Val: loss = 1.22147, acc = 0.51220
Training: loss = 1.30222, acc = 0.48427 | Val: loss = 1.22139, acc = 0.51220
Training: loss = 1.22179, acc = 0.51224 | Val: loss = 1.22143, acc = 0.51220
Training: loss = 1.29335, acc = 0.51049 | Val: loss = 1.22079, acc = 0.51220
Training: loss = 1.26199, acc = 0.49825 | Val: loss = 1.22066, acc = 0.51220
Training: loss = 1.24816, acc = 0.50350 | Val: loss = 1.22032, acc = 0.51220
Training: loss = 1.27510, acc = 0.49301 | Val: loss = 1.22032, acc = 0.51220
Training: loss = 1.21877, acc = 0.51399 | Val: loss = 1.22116, acc = 0.51220

Training: loss = 1.33896, acc = 0.52098 | Val: loss = 1.19852, acc = 0.51220
Training: loss = 1.36983, acc = 0.51224 | Val: loss = 1.20132, acc = 0.51220
Training: loss = 1.20374, acc = 0.54196 | Val: loss = 1.20477, acc = 0.51220
Training: loss = 1.25355, acc = 0.51573 | Val: loss = 1.20743, acc = 0.51220
Training: loss = 1.18218, acc = 0.56993 | Val: loss = 1.21043, acc = 0.51220
Training: loss = 1.27286, acc = 0.51748 | Val: loss = 1.21326, acc = 0.50000
Training: loss = 1.22384, acc = 0.55070 | Val: loss = 1.21666, acc = 0.50000
Training: loss = 1.26885, acc = 0.47902 | Val: loss = 1.21917, acc = 0.50000
Training: loss = 1.26200, acc = 0.50699 | Val: loss = 1.22154, acc = 0.50000
Training: loss = 1.22632, acc = 0.54371 | Val: loss = 1.22414, acc = 0.50000
Training: loss = 1.29936, acc = 0.50874 | Val: loss = 1.22525, acc = 0.50000
Training: loss = 1.16298, acc = 0.54545 | Val: loss = 1.22556, acc = 0.50000
Training: loss = 1.31371, acc = 0.51224 | Val: loss = 1.22453, acc = 0.50000